In [1]:
import cv2
import numpy as np
import pandas as pd
# from PIL import Image as im
# from IPython import display
from tkinter import *
from tkinter import filedialog , PhotoImage 
import os
import cv2
import tkinter as tk
from PIL import Image as im
from PIL import Image, ImageTk


In [2]:
classeFile = 'CocoNames.txt'
classesNames=[]
with open(classeFile , 'rt') as f :
    classesNames = f.read().rstrip('\n').split('\n')

In [3]:
modelConfig = 'yolov3.cfg'
modelWeights = 'yolov3.weights'
# create Network
net = cv2.dnn.readNetFromDarknet(modelConfig , modelWeights)
# use Opencv as the backend , and CPU
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
def detectt_video(video_Path) :
    cap = cv2.VideoCapture(video_Path)
     # cap width and height
    #frame_width = int(cap.get(3))
    #frame_height = int(cap.get(4))
   

    #videoWriter object (copy the frames in outputVd)
    #out = cv2.VideoWriter(save_path , cv2.VideoWriter_fourcc('m','p','4','v'), 10, (frame_width,frame_height))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame=cv2.resize(frame,((520,290)))
        

        if (cv2.waitKey(55) == ord('q')):
            break  
        blob = cv2.dnn.blobFromImage(frame , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)
        net.setInput(blob)
        LayersNames =net.getLayerNames()
        IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1)
        outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
        outputs = net.forward(outLayersNames)
#       ------------ Evoque methode --------------
        #findObjects(outputs , img)
        bbox , classesIds , confs , indicesKeeping = findObjects(outputs , frame)[:]
        indicesKeeping = cv2.dnn.NMSBoxes(bbox , confs , score_threshold = 0.8 , nms_threshold = 0.3 ) 
        for i in indicesKeeping :
            TrueBbox = bbox[i]
            x , y , w , h =TrueBbox[:] 
    #         draw the bbox as rectangle 
            cv2.rectangle(frame , (x , y) , (x+w , y+h) , (255 ,0 ,255) , 2)
            cv2.putText(frame , " {} {}%".format(classesNames[classesIds[i]] ,int(confs[i]*100)) , (x , y-15),
                        cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,(255 , 0,255 ) , 2)
#         plot figure : 
            #out.write(frame)
            cv2.imshow("video" , frame)
    #out.release() 
    cap.release()
    cv2.destroyAllWindows()

In [5]:
import cv2

# ouverture de la vidéo source
cap = cv2.VideoCapture('people.mp4')

# déterminer les propriétés de la vidéo source
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS)) # frame par seconde

# création de l'objet VideoWriter
out = cv2.VideoWriter('video_sortie.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width,frame_height))

# boucle pour lire les frames de la vidéo source et écrire les frames dans la vidéo de sortie
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # écrire le frame dans la vidéo de sortie
        out.write(frame)
        # affichage du frame si nécessaire
        cv2.imshow('frame',frame)
        # interrompre la boucle si 'q' est pressé
        if cv2.waitKey(2)==ord('q'):
            break
    else:
        break

# libération des ressources et fermeture des fenêtres
cap.release()
out.release()
cv2.destroyAllWindows()
print(out)


< cv2.VideoWriter 0000025762E12650>


In [6]:
v_conf = 0.5 # pour verifier l'existance de l'object 
def findObjects(outputs , img) :
    print("normal img" , img)
    hT , wT , cT = img.shape
    bbox = []
    classesIds = []
    confs = []
#     print("output" , outputs)
    for i in outputs :
        for j in i :
            scores = j[5:] # les probats des classes
            classId = np.argmax(scores) 
            confidence = scores[classId]
            if confidence > 0.5 :
                w , h = int(j[2]*wT) , int(j[3]*hT) 
                x , y = int((j[0]*wT)-w/2) , int((j[1]*hT) -h/2) 
                bbox.append([x , y , w , h])
                classesIds.append(classId)
                confs.append(float(confidence))
    indicesKeeping = cv2.dnn.NMSBoxes(bbox , confs , v_conf , nms_threshold = 0.3 ) # return indecices u should be keeped(more bbox decrease nmsValue)
    return [bbox , classesIds , confs ,indicesKeeping ]

In [7]:
def defaultPath(video_path):
#     print("avant : " , path)
    c = video_path.split("/")
#     print("apres split fonc : ", c)
    SeparerMot = c[-1].split('.')
    NewName = SeparerMot[0] + "detecter"+"."+SeparerMot[-1]
    c.remove(c[-1])
    autoSavePath = "/".join(c) + "/"+ NewName
    return autoSavePath

In [8]:
def detect_video():    
    def detect():
        path=filedialog.askopenfilename(initialdir = "F:/dell/Desktop/PFE/videoPath",filetypes=(("MP4 file","*.mp4"),("All Files","*.*")))
        global TruthPathSave
        print(defaultPath(path))
        #os.remove(defaultPath(path))
        dirictory = filedialog.askdirectory(title="Sélectionnez un répertoire de destination pour l'enregistrement" , mustexist = True)
        print('you choice : ' , dirictory)
#         Sélectionnez un répertoire de destination
        Selectnom = tk.simpledialog.askstring(Container , "choose name to video : " )
        print("name chosed " , Selectnom)
        while(Selectnom == "" or Selectnom.isspace()):
            Selectnom = tk.simpledialog.askstring(Container , "choose name to video *the name must be string*: " )
            print(Selectnom)
        TruthPathSave = dirictory+"/"+Selectnom+".mp4"
    
        #vid=detectt_video(path)
   

   
        #global pathdir
        #pathdir = ''

        
       
    #  --------------- Recording video ---------------------------
        # video writter object
        capt = cv2.VideoCapture(path)
        frame_width = int(capt.get(3))
        frame_height = int(capt.get(4))
        outFrame = cv2.VideoWriter(TruthPathSave , cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 10, (frame_width,frame_height))
        bbox_obj = []
        while(capt.isOpened()):
            if (cv2.waitKey(1)== ord('q')):
                break
            ret , FRAME = capt.read()
            FRAME=cv2.resize(FRAME,(520,290))
            
#             print("return" ,ret , "FRAME" , FRAME)
            # ---------------------------------
            blob = cv2.dnn.blobFromImage(FRAME , 1/255 , (320 , 320) , [0,0,0] , 1 , crop = False)
            net.setInput(blob)
            LayersNames =net.getLayerNames()
            IndexOutLayers = np.array(net.getUnconnectedOutLayers()).reshape(-1 , 1)
            outLayersNames = [LayersNames[i[0]-1] for i in IndexOutLayers]
            outputs = net.forward(outLayersNames)
#             print("Outputs ----" , outputs) run well
#             print("find funct result : "  ,findObjects(outputs , FRAME)[:])
            bbox , classesIds , confs , indicesKeeping = findObjects(outputs , FRAME)[:]
            print("bbox" , bbox , "\nClass id " , classesIds, "\nconfs" , confs , "\n indices keepin", indicesKeeping)
#             boite_index = 0
            for i in indicesKeeping :
                bbox_obj.append(bbox[i])
                TrueBbox = bbox[i]
                x , y , w , h =TrueBbox[:]
                cv2.rectangle(FRAME , (x , y) , (x+w , y+h) , (255 ,0 ,255) , 2)
                cv2.putText(FRAME , " {} {}%".format(classesNames[classesIds[i]] ,int(confs[i]*100)) , (x , y-15),
                            cv2.FONT_HERSHEY_SIMPLEX , 0.6 ,(255 , 0,255 ) , 2)
#                 if(((x - width/2) <= X_clique <= (x + width/2 )) and ((y - height/2 ) <= Y_clique <= (y+height/2))) :
#                     toSpeech.say(classesNames[classesIds[boite_index]])
#                     toSpeech.runAndWait()                          
#                 boite_index = boite_index+1    
#  --------------------- speech ---------------------
            outFrame.write(FRAME)
            cv2.imshow('webcam' , FRAME)
        outFrame.release()
        capt.release()
        cv2.destroyAllWindows()



    # videoWriter object (copy the frames in outputVd)
    
        #fps = int(cap.get(cv2.CAP_PROP_FPS))
        #out = cv2.VideoWriter(TruthPathSave, cv2.VideoWriter_fourcc(*'MP4V'), fps, (520,290))
        #out = cv2.VideoWriter(TruthPathSave , cv2.VideoWriter_fourcc('m','p','4','v'), 10, (520,290))
        
     
        #tk.messagebox.showinfo(title = 'information' , message = "Video Saved" , parent=Container)
        
        

    # _________________________________________________________________________
    def showVideo():
        global path 
        path=filedialog.askopenfilename(initialdir = "F:/dell/Desktop/PFE/videoPath",filetypes=(("MP4 file","*.mp4"),("All Files","*.*")))
        vid=detectt_video(path)
        
       
        return vid
   
    def Quit_windows():
        root.destroy()

    # _________________________________________________________________________
    root=Tk()
    titre = Label(root , text="Video Detection" , font=("Arial" , 17 , 'bold'),fg='white' , bg='black')
    titre.pack(fill=X)  
    quiticon = PhotoImage(file = 'F:/dell/Desktop/PFE/icons/quit.png')
#  
    frm= Frame(root , bg="#0C2864")
    frm.place(x=100 , y=400 , width=70, height=70)
    frm.pack(side= BOTTOM, padx=50, pady=50)
    quitterbtn = Button(frm , text = "Quitter  " ,image=quiticon,compound=LEFT ,bg='red',font =('Arial',15), cursor="hand2" , fg='white' , command=Quit_windows)
    quitterbtn.pack(side=BOTTOM,padx=3)
    # _________________________________________________________________________
    # ---------  frame 1 : container ------
    global Container
    Container = Frame(root , bg='#264d84')
    Container.place(x=100 , y=50 , width=810, height=500)
    
    # _________________________________________________________________________
    #   ---------  frame : video detecter  ------
    global frameVidDetect
    frameVidDetect = Frame(root , bg='white')
    frameVidDetect.place(x=120 , y=80 , width=770, height=460) 
    texte2 = Label(frameVidDetect , text=' video Detect ', font=("Arial" , 16 , 'bold'),fg='white',bg='#0C2864')
    texte2.pack(fill=X )
    lblDetec= Label(frameVidDetect ,bg="white")
    lblDetec.pack(pady=60) 
    # _________________________________________________________________________
    # btn1 ____________________________________________
    icon1 = PhotoImage(file = "F:/dell/Desktop/PFE/icons/import.png")
    importIcon = icon1.subsample(15,15) # resizes the image represented by icon1 by reducing its size
    btn1_img=Button(frm, text=" Detect video ", bg='beige',font =('Arial',15), cursor="hand2", image=importIcon,compound =LEFT, command=showVideo)#aaff66
    btn1_img.pack(side=BOTTOM, pady=5)
    # btn2_______________________________________________
    icon1 = PhotoImage(file = "F:/dell/Desktop/PFE/icons/detection.png")
    detectionIcon = icon1.subsample(15,15)
    
    btn2_img= Button(frm,text=" Detect and registre video " , bg='green',font =('Arial',15),cursor="hand2" ,image = detectionIcon ,compound=LEFT ,command=detect)
    btn2_img.pack(side=BOTTOM, padx=30)
    # _________________________________________________________________________
    # -------------------------- Windows ----------------------------
    root.title("Video Detection")
    root.geometry("1000x900")
    root.resizable(False , True)
#     root.iconbitmap("./icons/16.png")
    root.configure(bg='#0C2864')#configure() allows you to change the properties of the window.
    root.mainloop()
    

In [ ]:
detect_video()

normal img [[[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [104 154 182]
  [103 152 183]
  [120 169 200]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 83 113 158]
  [ 79 109 157]
  [ 92 121 169]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [109 110 184]
  [109 109 184]
  [103 103 178]]

 ...

 [[ 23  33  45]
  [ 23  33  45]
  [ 23  33  45]
  ...
  [ 90 104 113]
  [ 92 109 118]
  [ 96 113 122]]

 [[ 23  33  45]
  [ 23  33  45]
  [ 23  33  45]
  ...
  [ 91 104 114]
  [ 91 108 117]
  [ 93 110 119]]

 [[ 23  33  45]
  [ 23  33  45]
  [ 23  33  45]
  ...
  [ 91 105 114]
  [ 91 108 117]
  [ 91 108 117]]]
normal img [[[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [110 160 188]
  [111 160 191]
  [125 174 205]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 90 128 166]
  [ 84 122 163]
  [ 99 137 177]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 95 103 168]
  [ 95 103 171]
  [ 91  99 167]]

 ...

 [[ 18  28  40]
  [ 16  26  35

normal img [[[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 97 188 174]
  [ 97 188 174]
  [ 97 188 174]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 97 188 174]
  [ 97 188 174]
  [ 97 188 174]]

 [[ 46  67  32]
  [ 46  67  32]
  [ 46  67  32]
  ...
  [ 97 188 174]
  [ 97 188 174]
  [ 97 188 174]]

 ...

 [[ 30  27  77]
  [ 30  27  78]
  [ 28  27  78]
  ...
  [126 125 151]
  [126 125 151]
  [126 125 151]]

 [[ 33  29  81]
  [ 32  28  83]
  [ 31  28  83]
  ...
  [126 124 152]
  [126 124 152]
  [126 124 152]]

 [[ 37  32  88]
  [ 37  31  89]
  [ 35  30  90]
  ...
  [126 124 152]
  [126 124 152]
  [126 124 152]]]
